In [16]:
import pandas as pd
import numpy as np
from sklearn.base import BaseEstimator
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold, StratifiedKFold, cross_val_score, GridSearchCV, train_test_split

In [32]:
df = pd.read_csv('./pandas-numpy-practices/titanic_train.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [33]:
X_titanic_df = df.drop(['Survived', 'Cabin'], axis=1)
y_titanic_df = df['Survived']

In [39]:
X_titanic_df['Sex'] = X_titanic_df['Sex'].apply(lambda x: 0 if x=='female' else 1)

In [40]:
X_train, X_test, y_train, y_test = train_test_split(X_titanic_df,
                                                   y_titanic_df,
                                                   test_size=0.2,
                                                   random_state=11)

In [41]:
class DummyClassifier(BaseEstimator):
    def fit(self, X, y=None):
        pass
    
    def predict(self, X):
        pred = np.zeros((X.shape[0], 1))
        for i in range(X.shape[0]):
            # 예측 로직 = 남성이면 죽고, 여성이면 산다
            if X['Sex'].iloc[i] == 1:
                pred[i] = 0
            else:
                pred[i] = 1
                
        return pred

In [42]:
myclf = DummyClassifier()
myclf.fit(X_train, y_train)
pred = myclf.predict(X_test)

In [43]:
# 여자면 무조건 생존한다고 예측했을 때 생존여부를 65% 예측했음
accuracy_score(y_test, pred)

0.659217877094972

# 

In [29]:
from sklearn.datasets import load_digits

class MyFakeClassifier(BaseEstimator):
    def fit(self, X, y):
        pass
    def predict(self, X):
        return np.zeros((len(X), 1), dtype=bool)

digits = load_digits()

print(digits.data)
print(digits.target)

[[ 0.  0.  5. ...  0.  0.  0.]
 [ 0.  0.  0. ... 10.  0.  0.]
 [ 0.  0.  0. ... 16.  9.  0.]
 ...
 [ 0.  0.  1. ...  6.  0.  0.]
 [ 0.  0.  2. ... 12.  0.  0.]
 [ 0.  0. 10. ... 12.  1.  0.]]
[0 1 2 ... 8 9 8]


In [46]:
y = (digits.target == 7).astype(int)
y

array([0, 0, 0, ..., 0, 0, 0])

In [47]:
X_train, X_test, y_train, y_test = train_test_split(digits.data,
                                                   y,
                                                   random_state=11)

In [48]:
pd.Series(y_test).value_counts()

0    405
1     45
dtype: int64

In [49]:
fakeclf = MyFakeClassifier()
fakeclf.fit(X_train, y_train)
fake_pred = fakeclf.predict(X_test)
accuracy_score(y_test, fake_pred)

0.9

# Confusion Matrix
- True Negative : 실제로 negative인데 맞췄음
- False Positive : 실제로 positive인데 틀렸음
- False Negative : 실제로 negative인데 틀렸음
- True Positive : 실제로 positive인데 맞췄음

### Accuracy
**전체 데이터 중에서 positive/negative를 올바르게 예측한 비율**
$$\frac{tp+tn}{tp+tn+fp+fn}$$

### Precision = 정밀도
**Positive로 예측한 데이터 중에 진짜로 Positive인 비율**
- 실제 negative 음성인 데이터 예측을 positive 양성으로 잘못 판단하면 치명적인 경우
    - 스팸 메일

$$\frac{tp}{tp+fp}$$


### Recall = 재현율
**진짜 Positive인 데이터 중에 올바르게 예측한 비율**
- 실제 negative 양성인 데이터 예측을 negative로 잘못 판단하게 되면 치명적인 경우
    - 암 진단, 금융사기

$$\frac{tp}{tp+fn}$$

## Precision과 Recall은 trade-off 관계가 있음

In [51]:
from sklearn.metrics import confusion_matrix
cfm = confusion_matrix(y_test, fake_pred)
cfm

array([[405,   0],
       [ 45,   0]])

In [61]:
tn, fp, fn, tp = cfm.ravel()

In [62]:
tn, fp, fn, tp

(405, 0, 45, 0)

In [56]:
tn = cfm[0][0]
fp = cfm[0][1]
fn = cfm[1][0]
tp = cfm[1][1]

In [57]:
tn, fp, fn, tp

(405, 0, 45, 0)

In [63]:
# precision
tp / (tp+fp) # tp+fp=0으로 정의되지 않음

/opt/anaconda3/envs/AI_dev/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in long_scalars
  


nan

In [64]:
# recall
tp / (tp+fn)

0.0

## 정밀도(precision)와 재현율(recall)의 맹점

#### 정밀도 100%?
- 정밀도 = TP / (TP + FP)
- 확실한 기준이 되는 경우만 positive, 나머지는 모두 negative로 예측
- 전체 환자 1000명 중 확실한 환자가 1명
- 이 한 명만 positive로 예측하고 나머지는 모두 negative 판정
- FP = 0, TP = 1, 즉 정밀도는 1 / (1+0) = 100%가 됨

#### 재현율 100%?
- 재현율 = TP / (TP + FN)
- 전체 환자 1000명을 다 positive로 예측
- 실제 양성인 사람이 30명 정도라도 TN이 수치에 포함되지 않고
- FN은 0, 즉 재현율은 30/(30+0) = 100%


## F1 Score
- 정밀도와 재현율을 결합한 지표
- 정밀도와 재현율이 어느 한쪽으로 치우치지 않을 때 상대적으로 높은 값을 갖는다

$$\text{F1} = \frac{2}{\frac{1}{recall}+\frac{1}{precision}} = 2*\frac{precision*recall}{precision+recall}$$

- A모델 정밀도 0.9, 재현율 0.1 --- F1 Score 0.18
- B모델 정밀도 0.5, 재현율 0.5 --- F1 Score 0.5